# 数据预处理

In [1]:
import os
import pandas as pd
import cv2

print("env is done")

env is done


## 提取视频文件为帧图片（每秒截取一帧）

In [ ]:
ROOT_DIR = "autodl-tmp"
save_dir = './frames/' 
VIDEO_NAMES = os.listdir(os.path.join(ROOT_DIR, "videos"))
VIDEO_NAMES = sorted([x for x in VIDEO_NAMES if 'mp4' in x])

FRAME_NUMBERS = 0

for video_name in VIDEO_NAMES:
    path = os.path.join(ROOT_DIR, "videos", video_name)
    save_dirs = save_dir + video_name.replace('.mp4', '') +'/'
    save_dirs = os.path.join(ROOT_DIR, save_dirs)
    if os.path.exists(save_dirs):
        continue
    print(video_name)
    vidcap = cv2.VideoCapture(path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    print("fps", fps)
    if fps != 25:
        print(video_name, 'not at 25fps', fps)
    success=True
    count=0
    
    os.makedirs(save_dirs, exist_ok=True)
    while success is True:
        success,image = vidcap.read()
        if success:
            if count % fps == 0:
                cv2.imwrite(save_dirs + str(int(count//fps)).zfill(5) + '.png', image)
            count+=1
    vidcap.release()
    cv2.destroyAllWindows()
    print(count)
    FRAME_NUMBERS += count

print('Total Frams', FRAME_NUMBERS)

## 处理标注文件

In [ ]:
# 路径设置
path_annot = 'autodl-tmp/phase_annotations'
out_paths = 'autodl-tmp/annotations'

# 阶段名称到整数的映射
phase_to_int = {
    "Preparation": 0,
    "CalotTriangleDissection": 1,
    "ClippingCutting": 2,
    "GallbladderDissection": 3,
    "GallbladderPackaging": 4,
    "CleaningCoagulation": 5,
    "GallbladderRetraction": 6
}

def process_annotations(input_file, output_file, step=25):
    # 读取标注文件
    annotations = pd.read_csv(input_file, delim_whitespace=True, header=0)
    # 选择每隔25行的数据，并立即创建一个副本避免SettingWithCopyWarning
    selected_annotations = annotations.iloc[::step].copy()
    # 将阶段描述转换为整数
    selected_annotations['Phase'] = selected_annotations['Phase'].map(phase_to_int)
    # 保存到新的文本文件
    selected_annotations['Phase'].to_csv(output_file, index=False, header=False)

# 确保输出目录存在
os.makedirs(out_paths, exist_ok=True)

# 处理所有文件
input_files = os.listdir(path_annot)
for input_file in input_files:
    file_path = os.path.join(path_annot, input_file)
    # 修改输出文件名，移除 "-phase" 部分
    output_filename = input_file.replace('-phase.txt', '.txt')
    out_path = os.path.join(out_paths, output_filename)
    process_annotations(file_path, out_path)
    print(f"Processed: {output_filename}")